In [11]:
# !pip install tensorflow_addons

In [4]:
#!/usr/bin/env python3
#
# Team members' IDs:
# 182a3da8-8a9e-11ec-986f-f39926f24a9c  (Jan Zubáč)
# 7797f596-9326-11ec-986f-f39926f24a9c
# 449dba85-9adb-11ec-986f-f39926f24a9c
#
import argparse
import datetime
import os
import re
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import numpy as np
import tensorflow as tf

import argparse
import datetime
import os
import re
from typing import Dict
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import tensorflow_addons as tfa

from morpho_analyzer import MorphoAnalyzer
from morpho_dataset import MorphoDataset
from lemmatizer_noattn import Model


# TODO: Define reasonable defaults and optionally more parameters.
# Also, you can set the number of the threads 0 to use all your CPU cores.
# parser = argparse.ArgumentParser()

# parser.add_argument("--batch_size", default=64, type=int, help="Batch size.")
# parser.add_argument("--cle_dim", default=64, type=int, help="CLE embedding dimension.")
# parser.add_argument("--epochs", default=10, type=int, help="Number of epochs.")
# parser.add_argument("--max_sentences", default=None, type=int, help="Maximum number of sentences to load.")
# parser.add_argument("--recodex", default=False, action="store_true", help="Evaluation in ReCodEx.")
# parser.add_argument("--rnn_dim", default=64, type=int, help="RNN cell dimension.")
# parser.add_argument("--seed", default=42, type=int, help="Random seed.")
# parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

def main(args: argparse.Namespace) -> None:
    # Fix random seeds and threads
    tf.keras.utils.set_random_seed(args.seed)
    tf.config.threading.set_inter_op_parallelism_threads(args.threads)
    tf.config.threading.set_intra_op_parallelism_threads(args.threads)

    # Create logdir name
    args.logdir = os.path.join("logs", "{}-{}-{}".format(
        os.path.basename(globals().get("__file__", "notebook")),
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    ))

    # Load the data. Using analyses is only optional.
    morpho = MorphoDataset("czech_pdt_lemmas", add_bow_eow=True)
    analyses = MorphoAnalyzer("czech_pdt_analyses")

    # TODO: Create the model and train it
    # Create the model and train
    model = Model(args, morpho.train)

    # Construct dataset for lemmatizer training
    def create_dataset(name):
        dataset = getattr(morpho, name).dataset
        dataset = dataset.map(lambda example: (example["forms"], example["lemmas"]))
        dataset = dataset.shuffle(len(dataset), seed=args.seed) if name == "train" else dataset
        dataset = dataset.apply(tf.data.experimental.dense_to_ragged_batch(args.batch_size))
        return dataset
    train, dev, test = create_dataset("train"), create_dataset("dev"), create_dataset("test")


    class ShowIntermediateResults(tf.keras.callbacks.Callback):
        def __init__(self, data):
            self._iterator = iter(data.repeat())
        def on_train_batch_end(self, batch, logs=None):
            if model.optimizer.iterations % 50 == 0:
                forms, lemmas = next(self._iterator)
                print(model.optimizer.iterations.numpy(),
                      *[repr(strings[0, 0].numpy().decode("utf-8"))
                        for strings in [forms, lemmas, model.predict_on_batch(forms[:1, :1])]])



    logs = model.fit(train, epochs=args.epochs, validation_data=dev, verbose=2,
                    callbacks=[ShowIntermediateResults(dev), model.tb_callback])

    # Generate test set annotations, but in `args.logdir` to allow parallel execution.
    os.makedirs(args.logdir, exist_ok=True)
    with open(os.path.join(args.logdir, "lemmatizer_competition.txt"), "w", encoding="utf-8") as predictions_file:
        # Predict the tags on the test set; update the following prediction
        # command if you use other output structre than in lemmatizer_noattn.
        predictions = model.predict(test)
        for sentence in predictions:
            for word in sentence:
                print(word.numpy().decode("utf-8"), file=predictions_file)
            print(file=predictions_file)


# if __name__ == "__main__":
#     args = parser.parse_args([] if "__file__" not in globals() else None)
#     main(args)


In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=64, type=int, help="Batch size.")
parser.add_argument("--cle_dim", default=64, type=int, help="CLE embedding dimension.")
parser.add_argument("--epochs", default=10, type=int, help="Number of epochs.")
parser.add_argument("--max_sentences", default=None, type=int, help="Maximum number of sentences to load.")
parser.add_argument("--recodex", default=False, action="store_true", help="Evaluation in ReCodEx.")
parser.add_argument("--rnn_dim", default=64, type=int, help="RNN cell dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=64, cle_dim=64, epochs=10, max_sentences=None, recodex=False, rnn_dim=64, seed=42, threads=0)
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/bidirectional_2/forward_gru_2/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


50 '|' '|' ''
100 'První' 'první-1' 'ooot'
150 'Druhá' 'druhý' 'poot'
200 'Přesto' 'přesto-1' 'pooott'
250 'Do' 'do-1' 'do-1'
300 'Pozvání' 'pozvání' 'podavan'
350 'Už' 'už-2' 'že-1'
400 'Kdo' 'kdo' 'do-1'
450 'Nedávno' 'dávno-1' 'dnodný'
500 'Zcela' 'zcela' 'zeze'
550 'Tento' 'tento' 'tento'
600 'Zpočátku' 'zpočátku' 'provitný'
650 'Vbrzku' 'vbrzku' 'vruzko'
700 'Co' 'co-1' 'ok-1'
750 'Základní' 'základní' 'zalákdan'
800 'Podle' 'podle-2' 'podle'
850 'Zdá' 'zdát' 'zdám'
900 'Dále' 'dále-3' 'dále'
950 'Jistě' 'jistě' 'jistě'
1000 'V' 'v-1' 'v-1'
1050 'Na' 'na-1' 'na-1'
1100 'Kursy' 'kurs' 'Kurs'
1150 'Ctí' 'čest' 'cít'
1200 'Svět' 'svět' 'svět'
1250 'Zástupce' 'zástupce' 'zásture'
1300 'Gibbon' 'Gibbon' 'Gibobno'
1350 'Druhým' 'druhý' 'druhý'
1400 '"' '"' '"'
1417/1417 - 116s - loss: 1.2492 - val_accuracy: 0.6766 - 116s/epoch - 82ms/step
Epoch 2/10
1450 'Suková' 'Suková' 'Sukový'
1500 'Doporučujeme' 'doporučovat' 'doporubočat'
1550 'Dokončený' 'dokončený' 'dokončený'
1600 'Podle' 'podl

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=32, type=int, help="Batch size.")
parser.add_argument("--cle_dim", default=128, type=int, help="CLE embedding dimension.")
parser.add_argument("--epochs", default=10, type=int, help="Number of epochs.")
parser.add_argument("--max_sentences", default=None, type=int, help="Maximum number of sentences to load.")
parser.add_argument("--recodex", default=False, action="store_true", help="Evaluation in ReCodEx.")
parser.add_argument("--rnn_dim", default=128, type=int, help="RNN cell dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=32, cle_dim=128, epochs=10, max_sentences=None, recodex=False, rnn_dim=128, seed=42, threads=0)
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/bidirectional_4/forward_gru_4/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


50 '|' '|' ''
100 'Zkrácená' 'zkrácený' 'poooott'
150 'První' 'první-1' 'povov-1'
200 'Nejdříve' 'dříve' 'vevevení'
250 'Druhá' 'druhý' 'rrár'
300 'Znamená' 'znamenat' 'znantan'
350 'Přesto' 'přesto-1' 'přest'
400 'Při' 'při-1' 'při-1'
450 'Do' 'do-1' 'do-1'
500 'Abychom' 'aby' 'abych'
550 'Pozvání' 'pozvání' 'pozvání'
600 'Je' 'být' 'být'
650 'Už' 'už-2' 'už-1'
700 'Janáček' 'Janáček' 'Janáčký'
750 'Kdo' 'kdo' 'kdo'
800 'Překvapit' 'překvapit' 'překvařit'
850 'Nedávno' 'dávno-1' 'dávnout'
900 'Pak' 'pak' 'pak'
950 'Zcela' 'zcela' 'Zcela'
1000 'V' 'v-1' 'v-1'
1050 'Tento' 'tento' 'tento'
1100 'Pan' 'Pan-1' 'Pan'
1150 'Zpočátku' 'zpočátku' 'zpočátek'
1200 '9' '9' '9'
1250 'Vbrzku' 'vbrzku' 'vbrák'
1300 'Jako' 'jako-1' 'jako-1'
1350 'Co' 'co-1' 'co-1'
1400 'Siemens' 'Siemens-1' 'Siemens-1'
1450 'Základní' 'základní' 'základní'
1500 'Pozor' 'pozor-1' 'Pozor'
1550 'Podle' 'podle-2' 'podle-2'
1600 'V' 'v-1' 'v-1'
1650 'Zdá' 'zdát' 'zdát'
1700 'Akcionáři' 'akcionář' 'akcionář'
1750 'Dále' 'd

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=32, type=int, help="Batch size.")
parser.add_argument("--cle_dim", default=128, type=int, help="CLE embedding dimension.")
parser.add_argument("--epochs", default=25, type=int, help="Number of epochs.")
parser.add_argument("--max_sentences", default=None, type=int, help="Maximum number of sentences to load.")
parser.add_argument("--recodex", default=False, action="store_true", help="Evaluation in ReCodEx.")
parser.add_argument("--rnn_dim", default=128, type=int, help="RNN cell dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=32, cle_dim=128, epochs=25, max_sentences=None, recodex=False, rnn_dim=128, seed=42, threads=0)
Epoch 1/25


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/model_5/bidirectional_5/forward_gru_5/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


50 '|' '|' ''
100 'Zkrácená' 'zkrácený' 'poooott'
150 'První' 'první-1' 'povov-1'
200 'Nejdříve' 'dříve' 'vevevení'
250 'Druhá' 'druhý' 'rrár'
300 'Znamená' 'znamenat' 'znantan'
350 'Přesto' 'přesto-1' 'přest'
400 'Při' 'při-1' 'při-1'
450 'Do' 'do-1' 'do-1'
500 'Abychom' 'aby' 'abych'
550 'Pozvání' 'pozvání' 'pozvání'
600 'Je' 'být' 'být'
650 'Už' 'už-2' 'už-1'
700 'Janáček' 'Janáček' 'Janáčký'
750 'Kdo' 'kdo' 'kdo'
800 'Překvapit' 'překvapit' 'překvařit'
850 'Nedávno' 'dávno-1' 'dávnout'
900 'Pak' 'pak' 'pak'
950 'Zcela' 'zcela' 'Zcela'
1000 'V' 'v-1' 'v-1'
1050 'Tento' 'tento' 'tento'
1100 'Pan' 'Pan-1' 'Pan'
1150 'Zpočátku' 'zpočátku' 'zpočátek'
1200 '9' '9' '9'
1250 'Vbrzku' 'vbrzku' 'vbrák'
1300 'Jako' 'jako-1' 'jako-1'
1350 'Co' 'co-1' 'co-1'
1400 'Siemens' 'Siemens-1' 'Siemens-1'
1450 'Základní' 'základní' 'základní'
1500 'Pozor' 'pozor-1' 'Pozor'
1550 'Podle' 'podle-2' 'podle-2'
1600 'V' 'v-1' 'v-1'
1650 'Zdá' 'zdát' 'zdát'
1700 'Akcionáři' 'akcionář' 'akcionář'
1750 'Dále' 'd

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=128, type=int, help="Batch size.")
parser.add_argument("--cle_dim", default=192, type=int, help="CLE embedding dimension.")
parser.add_argument("--epochs", default=10, type=int, help="Number of epochs.")
parser.add_argument("--max_sentences", default=None, type=int, help="Maximum number of sentences to load.")
parser.add_argument("--recodex", default=False, action="store_true", help="Evaluation in ReCodEx.")
parser.add_argument("--rnn_dim", default=192, type=int, help="RNN cell dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=128, cle_dim=192, epochs=10, max_sentences=None, recodex=False, rnn_dim=192, seed=42, threads=0)
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_6/bidirectional_6/forward_gru_6/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


50 '|' '|' ''
100 'Druhá' 'druhý' 'prod'
150 'Do' 'do-1' 'od-1'
200 'Už' 'už-2' 'že-1'
250 'Nedávno' 'dávno-1' 'dávní'
300 'Tento' 'tento' 'tento'
350 'Vbrzku' 'vbrzku' 'výrzk'
400 'Základní' 'základní' 'základní'
450 'Zdá' 'zdát' 'zdát'
500 'Jistě' 'jistě' 'Jistě'
550 'Na' 'na-1' 'na-1'
600 'Ctí' 'čest' 'tít'
650 'Zástupce' 'zástupce' 'zástupce'
700 'Druhým' 'druhý' 'druhý'
709/709 - 88s - loss: 0.8805 - val_accuracy: 0.8633 - 88s/epoch - 124ms/step
Epoch 2/10
750 'Suková' 'Suková' 'Sukovat'
800 'Dokončený' 'dokončený' 'dokončený'
850 'Jisté' 'jistý' 'jistý'
900 'Vykonávání' 'vykonávání' 'vykonávání'
950 'Radost' 'radost' 'radost'
1000 'Darja' 'Darja' 'darja'
1050 'Jižní' 'jižní' 'jižní'
1100 'Jím' 'on-1' 'jít'
1150 'Ve' 'v-1' 'v-1'
1200 'Pacient' 'pacient' 'pacient'
1250 'Jejich' 'jeho' 'jeho'
1300 'Potlačily' 'potlačit' 'potlačit'
1350 'Věda' 'věda' 'věda'
1400 'Jiří' 'Jiří' 'Jiří'
709/709 - 79s - loss: 0.0939 - val_accuracy: 0.9251 - 79s/epoch - 112ms/step
Epoch 3/10
1450 'Obě' 'ob

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=128, type=int, help="Batch size.")
parser.add_argument("--cle_dim", default=192, type=int, help="CLE embedding dimension.")
parser.add_argument("--epochs", default=25, type=int, help="Number of epochs.")
parser.add_argument("--max_sentences", default=None, type=int, help="Maximum number of sentences to load.")
parser.add_argument("--recodex", default=False, action="store_true", help="Evaluation in ReCodEx.")
parser.add_argument("--rnn_dim", default=192, type=int, help="RNN cell dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=128, cle_dim=192, epochs=25, max_sentences=None, recodex=False, rnn_dim=192, seed=42, threads=0)
Epoch 1/25


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_7/bidirectional_7/forward_gru_7/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


50 '|' '|' ''
100 'Druhá' 'druhý' 'prod'
150 'Do' 'do-1' 'od-1'
200 'Už' 'už-2' 'že-1'
250 'Nedávno' 'dávno-1' 'dávní'
300 'Tento' 'tento' 'tento'
350 'Vbrzku' 'vbrzku' 'výrzk'
400 'Základní' 'základní' 'základní'
450 'Zdá' 'zdát' 'zdát'
500 'Jistě' 'jistě' 'Jistě'
550 'Na' 'na-1' 'na-1'
600 'Ctí' 'čest' 'tít'
650 'Zástupce' 'zástupce' 'zástupce'
700 'Druhým' 'druhý' 'druhý'
709/709 - 89s - loss: 0.8805 - val_accuracy: 0.8632 - 89s/epoch - 126ms/step
Epoch 2/25
750 'Suková' 'Suková' 'Sukovat'
800 'Dokončený' 'dokončený' 'dokončený'
850 'Jisté' 'jistý' 'jistý'
900 'Vykonávání' 'vykonávání' 'vykonávání'
950 'Radost' 'radost' 'radost'
1000 'Darja' 'Darja' 'darja'
1050 'Jižní' 'jižní' 'jižní'
1100 'Jím' 'on-1' 'jít'
1150 'Ve' 'v-1' 'v-1'
1200 'Pacient' 'pacient' 'pacient'
1250 'Jejich' 'jeho' 'jeho'
1300 'Potlačily' 'potlačit' 'potlačit'
1350 'Věda' 'věda' 'věda'
1400 'Jiří' 'Jiří' 'Jiří'
709/709 - 80s - loss: 0.0939 - val_accuracy: 0.9250 - 80s/epoch - 113ms/step
Epoch 3/25
1450 'Obě' 'ob